# Subset Selection
## CMSE 381 - Fall 2023
## Oct 13,  2023. Lecture 17



In [1]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [2]:
# First, we're going to do all the data loading we've had for a while for this data set
auto = pd.read_csv('../../DataSets/Auto.csv')
auto = auto.replace('?', np.nan)
auto = auto.dropna()
auto.horsepower = auto.horsepower.astype('int')

#this shuffles my data set in advance so that i don't need to worry about it later 
auto = auto.sample(frac=1).reset_index(drop=True)


auto.head()


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,20.0,6,225.0,100,3651,17.7,76,1,dodge aspen se
1,14.0,8,304.0,150,4257,15.5,74,1,amc matador (sw)
2,22.0,6,225.0,100,3233,15.4,76,1,plymouth valiant
3,36.0,4,105.0,74,1980,15.3,82,2,volkswagen rabbit l
4,24.0,6,200.0,81,3012,17.6,76,1,ford maverick


Let's try to run subset selection on the `auto` data set! We're going to use `cylinders`, `horsepower`, `weight`, and `acceleration` to predict `mpg`. 

In [3]:
inputvars = ['cylinders','horsepower','weight', 'acceleration']

The first tool we are going to use is the `itertools` package, which gives us a way to get subsets of whatever size we want using the `combinations` command.  

In [4]:
from itertools import combinations

The weird thing is it's an iterator, so if I just try to print out what I want, it's not helpful to me. 

In [5]:
combinations(inputvars,2)

But if I use it in a for loop it does what I want!

In [6]:
for x in combinations(inputvars,2):
    print(x)

('cylinders', 'horsepower')
('cylinders', 'weight')
('cylinders', 'acceleration')
('horsepower', 'weight')
('horsepower', 'acceleration')
('weight', 'acceleration')


Here's some code stolen from the last few days to run linear regression on a subset of the input variables. 

In [17]:
def myscore(df,listofvars, outputvar = 'mpg'):
    X = df[list(listofvars)]
    y = df[outputvar]
    
    #build linear regression model
    model = LinearRegression()
    

    #use 5-fold CV to evaluate model
    scores = cross_val_score(model, X,y, 
                             scoring='neg_mean_squared_error',
                             cv=5)

    #view mean absolute error
    return np.average(np.absolute(scores))
    

myvars = ('cylinders', 'acceleration')
myscore(auto,myvars)

24.585455982088202


&#9989; **<font color=red>Do this:</font>** Modify the code below as follows: 
- Set up two nested for loops to get every size $p = \{1,\cdots,4\}$ subset of my list of variables I want to use
- For each of these subsets, use the `myscore` function to get the k-fold CV error.
- Append it into a data frame as shown


In [8]:
myvars = []
myscores = []


for size in range(1,4):
    for x in combinations(inputvars,size):
        myvars.append(x)
        myscores.append(myscore(auto,x))


myResults = pd.DataFrame({'Vars':myvars, 'Score':myscores})
myResults

,Vars,Score
0,"(cylinders,)",24.226169
1,"(horsepower,)",24.212157
2,"(weight,)",18.908031
3,"(acceleration,)",50.136065
4,"(cylinders, horsepower)",21.204128
5,"(cylinders, weight)",18.613476
6,"(cylinders, acceleration)",24.585456
7,"(horsepower, weight)",18.291244
8,"(horsepower, acceleration)",22.819141
9,"(weight, acceleration)",18.624851


We got all our main subsets, we're just missing a null model. This is the model that predicts the sample mean `mpg` for any input data. 

&#9989; **<font color=red>Do this:</font>** What is the MSE on our data set if we just predict the mean for every data point? Add this entry to your `myResults` data frame

*Hint: you can get a numpy array with every entry being the same output by using the `np.full` command.*

In [20]:
## Your code here ##


In [19]:
newresult = pd.DataFrame({'Vars':['empty'], 'Score':[myscore]})

myResults = pd.concat([newresult,myResults],ignore_index = True)

# If you print it out now, you should have 16 models scored
myResults

,Vars,Score
0,empty,<function myscore at 0x000001858FE5B1C0>
1,"(cylinders,)",24.226169
2,"(horsepower,)",24.212157
3,"(weight,)",18.908031
4,"(acceleration,)",50.136065
5,"(cylinders, horsepower)",21.204128
6,"(cylinders, weight)",18.613476
7,"(cylinders, acceleration)",24.585456
8,"(horsepower, weight)",18.291244
9,"(horsepower, acceleration)",22.819141


&#9989; **<font color=red>Do this:</font>** What is the minimum score over all subsets? Which model makes it happen? The `idxmin` command will likely be useful for this. 

In [ ]:
# Your code here #

## Stretch project

Have some free time? 

&#9989; **<font color=red>Strech project:</font>** Figure out how to do this for forward selection. 

- First, try to write pseudo code for how to search through these subsets. 
- Then if you still have time, see if you can get it working on this set of four variables. 

In [ ]:
# Your code here #



-----
### Congratulations, we're done!
Written by Dr. Liz Munch, Michigan State University

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.